# Borrower Reliability Research

The customer is the credit department of the bank. It is necessary to find out whether the marital status and the number of children of the client affect the fact of repaying the loan on time. Input data from the bank - statistics on the solvency of customers.

The results of the research will be taken into account when building a `credit scoring` model - a special system that assesses the ability of a potential borrower to repay a loan to a bank.

## 1. Open the data file and examine the general information

In [1]:
# import libraries
import pandas as pd
from pymystem3 import Mystem
import numpy as np
m = Mystem()

In [2]:
data = pd.read_csv('data.csv')
display(data.head())
data.info()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Сonclusion**
  - Opened and viewed the dataset. 
  - There are no problems with the column names, there are missing values in `days_employed` and `total_income`, we will replace them with the median value, and make the `days_employed` values positive. 
  - Also in these lines, `float` data, replace with `int`.

## 2. Data preprocessing:

### 2.1 `NaN` processing

In [3]:
data['days_employed'] = data['days_employed'].fillna(data['days_employed'].median())
data['total_income'] = data['total_income'].fillna(data['total_income'].median())
data['days_employed'] = abs(data['days_employed'])
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


**Сonclusion**
  - Replaced the missing values in `days_employed` and `total_income` with the median value of these columns
  - Converted values to a positive number

### 2.2 Replacing the data type

In [4]:
data = data.astype({"days_employed": "int64", "total_income": "int64"})
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21525 non-null  int64 
 1   days_employed     21525 non-null  int64 
 2   dob_years         21525 non-null  int64 
 3   education         21525 non-null  object
 4   education_id      21525 non-null  int64 
 5   family_status     21525 non-null  object
 6   family_status_id  21525 non-null  int64 
 7   gender            21525 non-null  object
 8   income_type       21525 non-null  object
 9   debt              21525 non-null  int64 
 10  total_income      21525 non-null  int64 
 11  purpose           21525 non-null  object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


**Сonclusion**
  - Replaced column data with int, since it is better to represent the number of days and total income as integers.

### 2.3 Handling Duplicate Values & outliers

In [5]:
data['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

There are outliers here: 
  - `-1` and `20` children.
  - We replace `-1` with `1` and delete data with `20` children (about 0.3%)


In [6]:
data.loc[data['children'] == -1,'children'] = 1
data = data[data.children != 20]

In [7]:
data['dob_years'].value_counts()

35    615
40    605
41    605
34    600
38    597
42    594
33    579
39    572
31    558
36    553
44    545
29    543
30    537
48    537
37    533
50    511
43    511
32    508
49    505
28    503
45    494
27    491
52    483
56    482
47    480
54    478
46    472
58    461
57    459
53    458
51    447
59    442
55    442
26    407
60    376
25    356
61    354
62    351
63    269
64    264
24    263
23    253
65    194
22    183
66    183
67    167
21    110
0     100
68     99
69     84
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64

There are outliers here: 
  - `0` dob_years - 100 (about 0.4%), we delete them.

In [8]:
data = data[data.dob_years != 0]

In [9]:
data['education'].value_counts()

среднее                13640
высшее                  4674
СРЕДНЕЕ                  768
Среднее                  702
неоконченное высшее      664
ВЫСШЕЕ                   271
Высшее                   266
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64

There are dublicates here: 
  - duplicates written in different case

In [10]:
data.education = data.education.str.lower()
data.education.value_counts()

среднее                15110
высшее                  5211
неоконченное высшее      740
начальное                282
ученая степень             6
Name: education, dtype: int64

In [11]:
data.family_status.value_counts()

женат / замужем          12283
гражданский брак          4144
Не женат / не замужем     2788
в разводе                 1183
вдовец / вдова             951
Name: family_status, dtype: int64

In [12]:
data.gender.value_counts()

F      14118
M       7230
XNA        1
Name: gender, dtype: int64

In [13]:
data.income_type.value_counts()

сотрудник          11022
компаньон           5043
пенсионер           3827
госслужащий         1451
безработный            2
предприниматель        2
студент                1
в декрете              1
Name: income_type, dtype: int64

In [14]:
data.debt.value_counts()

0    19624
1     1725
Name: debt, dtype: int64

Everything is fine here

In [15]:
data.purpose.value_counts()

свадьба                                   791
на проведение свадьбы                     769
сыграть свадьбу                           765
операции с недвижимостью                  672
покупка коммерческой недвижимости         658
покупка жилья для сдачи                   650
операции с коммерческой недвижимостью     645
операции с жильем                         643
покупка жилья для семьи                   639
покупка жилья                             638
жилье                                     637
недвижимость                              629
операции со своей недвижимостью           629
строительство собственной недвижимости    627
строительство жилой недвижимости          622
строительство недвижимости                620
покупка своего жилья                      619
покупка недвижимости                      619
ремонт жилью                              608
покупка жилой недвижимости                602
на покупку своего автомобиля              502
заняться высшим образованием      

Repeating categories, later we will combine them using lemmatization and categorization

In [16]:
data = data.drop_duplicates()
data.duplicated().sum()

0

Remove obvious duplicates

### 2.4 Lemmatization:

In [17]:
purpose_data = data['purpose'].value_counts()
lemmas = []
for x in data['purpose'].head(50):
    lemm = m.lemmatize(x)
    for l in lemm:
        if l not in lemmas:
            lemmas.append(l)

   - lemmatize the goal lemma of credits

In [18]:
lemmas

['покупка',
 ' ',
 'жилье',
 '\n',
 'приобретение',
 'автомобиль',
 'дополнительный',
 'образование',
 'сыграть',
 'свадьба',
 'операция',
 'с',
 'на',
 'проведение',
 'для',
 'семья',
 'недвижимость',
 'коммерческий',
 'жилой',
 'строительство',
 'собственный',
 'подержать',
 'свой',
 'со',
 'заниматься',
 'сделка',
 'подержанный',
 'получение']

In [19]:
# У нас получилось 4 цели:
house = 'жиль'
house_2 = 'недвиж'
auto = 'автомобил'
study = 'образов'
wedding = 'свадьб'

   - Received 4 categories of loan objectives

### 2.5 Categorization:

In [20]:
def purpose_categoryze(purpose):
    if house in purpose or house_2 in purpose:
        return 'property'
    elif auto in purpose:
        return 'car'
    elif study in purpose:
        return 'education'
    elif wedding in purpose:
        return 'wedding'
    
data['purpose_category'] = data['purpose'].apply(purpose_categoryze)
data['purpose_category'].value_counts()   

property     10728
car           4269
education     3980
wedding       2301
Name: purpose_category, dtype: int64

   - We divided the data of the `purpose` column into 4 categories

In [21]:
# Let's categorize "total_income" for further use by creating a new column "total_income_cat"

print(f'Max total_income: {data.total_income.max()}')
print(f'Min total_income: {data.total_income.min()}')
print(f'Median total_income: {data.total_income.median()}')

Max total_income: 2265604
Min total_income: 20667
Median total_income: 145017.0


In [22]:
# We looked at the maximum and minimum income, from 20,000 to about 2,000,000, the median is 145,000. 
# Most likely, 2 million this is an outlier, let's divide it into 3 categories:
    
def income_categoryze(value):
    if value < 100000:
        return 'small'
    elif 100000 <= value <= 200000:
        return 'average'
    else:
        return 'high'

data['total_income_cat'] = data['total_income'].apply(income_categoryze)
data['total_income_cat'].value_counts()

average    11829
high        5020
small       4429
Name: total_income_cat, dtype: int64

   - After creating and applying the function, we categorized the goals of the loan, highlighting 4 types,creating a separate column. 
   - Then did the same with income categories, dividing them into 3 groups. 
   - Checked everything using value_counts

## 3. Answer the questions according to the condition:

### 3.1 Is there a relationship between having children and repaying the loan on time?

In [23]:
debt_child = pd.DataFrame()
debt_child['count_child'] = data.groupby('children')['debt'].count()    
debt_child['sum_child'] = data.groupby('children')['debt'].sum()
debt_child['result_child'] = debt_child['sum_child'] / debt_child['count_child'] 
debt_child.sort_values('result_child', ascending = False)

debt_child

,count_child,sum_child,result_child
children,,,
0,14022,1058,0.075453
1,4839,442,0.091341
2,2039,194,0.095145
3,328,27,0.082317
4,41,4,0.097561
5,9,0,0.000000


<div class="alert alert-block alert-info"> Answer: We checked the dependence, we see that those who have children are more likely to overdue the loan than the childless.Conclusion - there is a dependence, judging by this sample </div>

### 3.2 Is there a relationship between marital status and loan repayment on time?

In [24]:
debt_fam = pd.DataFrame()
debt_fam['count_family'] = data.groupby('family_status')['debt'].count()
debt_fam['sum_family'] = data.groupby('family_status')['debt'].sum()
debt_fam['result_family'] = debt_fam['sum_family'] / debt_fam['count_family']
debt_fam

,count_family,sum_family,result_family
family_status,,,
Не женат / не замужем,2785,272,0.097666
в разводе,1183,84,0.071006
вдовец / вдова,950,62,0.065263
гражданский брак,4118,383,0.093006
женат / замужем,12242,924,0.075478


<div class="alert alert-block alert-info">Answer: Grouped by marital status, and looked at the dependence. Conclusion, unmarried / unmarried, or living in a civil marriage are more likely to delay the payment than people who are currently married or have been married before</div>

### 3.3 Is there a relationship between income level and loan repayment on time?

In [25]:
debt_income = pd.DataFrame()
debt_income['count_income'] = data.groupby('total_income_cat')['debt'].count()
debt_income['sum_income'] = data.groupby('total_income_cat')['debt'].sum()
debt_income['result_income'] = debt_income['sum_income'] / debt_income['count_income']
debt_income

,count_income,sum_income,result_income
total_income_cat,,,
average,11829,1017,0.085975
high,5020,355,0.070717
small,4429,353,0.079702


<div class="alert alert-block alert-info">Answer: People with average or low incomes are more likely to be in arrears than people with high incomes.</div>

### 3.4 How do different purposes of a loan affect its repayment on time?

In [26]:
debt_purpose = pd.DataFrame()
debt_purpose['count_purpose'] = data.groupby('purpose_category')['debt'].count()
debt_purpose['sum_purpose'] = data.groupby('purpose_category')['debt'].sum()
debt_purpose['result_purpose'] = debt_purpose['sum_purpose'] / debt_purpose['count_purpose']
debt_purpose

,count_purpose,sum_purpose,result_purpose
purpose_category,,,
car,4269,398,0.093230
education,3980,369,0.092714
property,10728,777,0.072427
wedding,2301,181,0.078661


<div class="alert alert-block alert-info">Answer: Those who take a loan for a car or for education are more likely to overdue.  </div>

## 4. General conclusion:

In this project, we have gone all the way:
   - opening a dataset,
   - processing, 
   - removing duplicates, 
   - categorizing and answering questions

The general conclusion can be drawn as follows:
   - By marital status - married or previously married allow fewer delays than unmarried or in a civil marriage.
   - People with children allow delays more often, apparently this is due to the fact that children are an additional expense item.
   - High income earners are less likely to be in arrears.

The most unsuccessful client for the bank (or vice versa successful) is an unmarried person with 4 children with a low income who took out a loan for a car and driving lessons.
